In [11]:
from sklearn import datasets, linear_model, neighbors, svm, ensemble
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from base import SuperLearner
from base import BMA
import pandas as pd
import numpy as np
# from pyearth import Earth

v_folds = 5
ols = linear_model.LinearRegression()
elnet = linear_model.ElasticNetCV(l1_ratio=0.5, cv=v_folds, normalize=True)
ridge = linear_model.RidgeCV(cv=v_folds)
lars = linear_model.LarsCV(cv=v_folds, normalize=True)
lasso = linear_model.LassoCV(cv=v_folds, normalize=True)
nn = neighbors.KNeighborsRegressor(weights='distance')
svm1 = svm.SVR(kernel='linear', C=10, gamma='auto')
svm2 = svm.SVR(kernel='poly', C=10, gamma='auto')
rf = ensemble.RandomForestRegressor(n_estimators=50)
model_lib = [ols, rf ,elnet, ridge, lars, lasso, nn, svm1, svm2]
model_names = ["OLS", "RF", "ElasticNet", "Ridge", "LARS", "LASSO", "kNN", "SVM rbf", "SVM poly"]
meta_learner = ols

In [12]:
sup=SuperLearner(cand_learners=[ols,lars,rf],V=10)

In [13]:
def sim1(n):
    w=np.random.binomial(1,.4,size=(10,n))
    eps=np.random.normal(0,1,size=n)
    y=2*w[0]*w[9]+4*w[1]*w[6]+3*w[3]*w[4]-\
    5*w[5]*w[9]+3*w[7]*w[8]+w[0]*w[1]*w[3]-\
    2*w[6]*(1-w[5])*w[1]*w[8]-4*(1-w[9])*w[0]*(1-w[3])+eps
    return np.transpose(w),y

In [16]:
train1=sim1(500)
sup.fit(*train1)
test1=sim1(10000)
df=sup.debug(*train1,*test1)
df

,name,training mse,testing mse,coef
0,LinearRegression,6.613248,6.177968,0.960733
1,LarsCV,6.613474,6.177167,-0.894419
2,RandomForestRegressor,0.499204,1.971300,0.958122
3,meta,0.534064,1.941747,NaN
